In [3]:
with open("Harry_potter1.txt","r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])


Total number of character: 439742
Harry Potter and the Sorcerer's Stone


CHAPTER ONE

THE BOY WHO LIVED

Mr. and Mrs. Dursley, of nu


In [4]:
import re

text = "Hello world. Thia is great fictional book of Harry Potter"
result = re.split(r'(\s)',text)
print(result)

['Hello', ' ', 'world.', ' ', 'Thia', ' ', 'is', ' ', 'great', ' ', 'fictional', ' ', 'book', ' ', 'of', ' ', 'Harry', ' ', 'Potter']
